In [ ]:
import os
import time
import random
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import ShuffleSplit
from slim_gsgp.main_mo_gp import mo_gp
from slim_gsgp.datasets.data_loader import (
     load_efficiency_cooling, load_ld50, 
     #load_boston, 
)

# Configuration
RANDOM_SEED = 37
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# Datasets to Test (we need to import them first)
DATASETS = {
    'Toxicity': load_ld50,
    'Cooling': load_efficiency_cooling, 
}


# Objective Configurations
OBJECTIVE_SETS = {
    '2_Objs': {
        'funcs': ["rmse", "size"],
        'flags': [True, True] # Min, Min
    },
    '3_Objs': {
        'funcs': ["rmse", "size", "features"],
        'flags': [True, True, True]
    },
    '5_Objs': {
        'funcs': ["rmse", "size", "features", "nao", "naoc"],
        'flags': [True, True, True, True, True]
    }
}

SCENARIOS = {
    'NSGA-II_Pure': {
        'selector': 'nsga2',
        'survival': 'nsga2',
        'n_elites': 0, 
        'elitism_strategy': 'nsga2'
    },
    'NT_NSGA-II': {
        'selector': 'nested_tournament',
        'survival': 'nsga2',
        'n_elites': 0,
        'elitism_strategy': 'nsga2'
    },
    'NT_No_Elitism': {
        'selector': 'nested_tournament',
        'survival': 'generational',
        'n_elites': 0,
        'elitism_strategy': 'nsga2'
    },
    'NT_1st_Obj_Elitism': {
        'selector': 'nested_tournament',
        'survival': 'generational',
        'n_elites': 1,
        'elitism_strategy': 'first_obj' #uses RMSE
    },
    'NT_Rank_CD_Elitism': {
        'selector': 'nested_tournament',
        'survival': 'generational',
        'n_elites': 1,
        'elitism_strategy': 'nsga2' #Rank + CD
    },
    'NT_Ideal_Cand_Elitism': {
        'selector': 'nested_tournament',
        'survival': 'generational',
        'n_elites': 1,
        'elitism_strategy': 'ideal_point' #dynamic ideal point
    }
}

#Hyperparameter Grid for Inner CV
FIXED_PARAMS = {
    'pop_size': 400,
    'n_iter': 250,
    'p_xo': 0.8,           #default  
    'prob_const': 0.2,     #default  
    'max_depth': 17,       #default  
    'initializer': 'rhh',  #default
    "init_depth": 4,
    "seed": 74,            #default
    "n_jobs": 1,           #default
    "test_elite": True    
}

#usado para teste pequeno
# FIXED_PARAMS = {
#     'pop_size': 10,
#     'n_iter': 5,
#     'p_xo': 0.8,
#     'prob_const': 0.2,    
#     'max_depth': 6,
#     'initializer': 'rhh', 
#     "init_depth": 6,      
#     "seed": 74,           
#     "n_jobs": 1,
#     "test_elite": True    
# }

##################################JUST COMMENTS##################################
#Other parameters will remain default:
###offspring_size###
# if offspring_size is None:
#     n_offspring = self.pop_size

###n_elites###
# it depends on the scenario being used

###log_path & log_level###
#posso criar um log =5 e fazer hard coded o que quero armazenar    APAGAR
# it depends on dataset, scenario, fold, hyperparams
# APAGAR: find out what information we need to log (its better to log more info and then ignore what is not needed than the opposite)

###fitness_functions, minimization_flags###
#it depends on the scenario being used: OBJECTIVE_SETS (Ok)

###tournament_sizes###
# Tournament sizes need to be dynamic based on n_objectives, handled in loop. When using Nested Tournament Selection, but for now we will fix it 

###ideal_candidate_values###
# it can no longer be user defined      

### "test_elite": True###

### tree_functions ###
# FUNCTIONS = {
#     'add': {'function': torch.add, 'arity': 2},
#     'subtract': {'function': torch.sub, 'arity': 2},
#     'multiply': {'function': torch.mul, 'arity': 2},
#     'divide': {'function': utils.protected_div, 'arity': 2},
#     'mod': {'function': utils.protected_mod, 'arity': 2},
#     'pow': {'function': utils.protected_pow, 'arity': 2},
# }

### tree_constants ###
# I changed what originaly was in gp_config.py to the following:
# random.seed(47)
# CONSTANTS = {
#     f'constant_{i}': lambda _, val=random.uniform(-1, 1): torch.tensor(val)
#     for i in range(10)
# }
#################################################################################


N_SPLITS_MC = 30 
TEST_SIZE = 0.3

<div style="background-color:#e5c120ff; padding:1px; border-radius:10px;">
</div>

In [22]:
# from sklearn.model_selection import ShuffleSplit
# TEST_SIZE = 0.3
#APAGAR

### Main Evaluation Loop

In [23]:
#For each dataset: load data
    # For each Outer fold: (14 folds for train + 1 fold for test each time) -> what will be use to track scores
        # For each objective structure (Obj: 2, 3, 5)
            # For each Scenario
                # For each combination of current_grid# Estrutura: 
    
#./log_mo/DATASET/SCENARIO/OBJECTIVES/fold_X.csv


In [24]:
if __name__ == "__main__":
    for ds_name, loader_func in DATASETS.items():
        print(f"\n{'='*40}\nDataset: {ds_name}\n{'='*40}")
    
        # Load Data (X: features, y: target)
        X, y = loader_func(X_y=True)
    
        # Monte Carlo CV (30 random splits)
        rs = ShuffleSplit(n_splits=N_SPLITS_MC, test_size=TEST_SIZE, random_state=RANDOM_SEED)
    
        for split_idx, (train_idx, test_idx) in enumerate(rs.split(X, y)):
            print(f"\n  > MC Split {split_idx+1}/{N_SPLITS_MC}")

            X_train = X[train_idx]
            y_train = y[train_idx]
            X_test = X[test_idx]
            y_test = y[test_idx]

            for obj_set_name, obj_config in OBJECTIVE_SETS.items():
                n_objs = len(obj_config['funcs'])
                print(f"    > Objectives: {obj_set_name}")
                
                for scen_name, scen_config in SCENARIOS.items():
                    # print(f"      > Scenario: {scen_name}")
                    
                    fold_dir = f"./log_mo/{ds_name}/{scen_name}/{obj_set_name}/fold_{split_idx+1}"
                    
                    if not os.path.exists(fold_dir): os.makedirs(fold_dir)
                    
                    log_path = os.path.join(fold_dir, "execution_log.csv")

                    #if we already ran it, skip
                    if os.path.exists(log_path):
                            continue
                    
                    t_sizes_final = [2] * n_objs

                    try:
                        mo_gp(
                            X_train=X_train, y_train=y_train,
                            X_test=X_test, y_test=y_test,
                            dataset_name=ds_name,
                            fitness_functions=obj_config['funcs'],
                            minimization_flags=obj_config['flags'],
                            tournament_sizes=t_sizes_final,
                            elitism_strategy=scen_config['elitism_strategy'],
                            selector_strategy=scen_config['selector'],
                            survival_strategy=scen_config['survival'],
                            n_elites=scen_config['n_elites'],
                                
                            #fixed params
                            pop_size=FIXED_PARAMS['pop_size'],
                            n_iter=FIXED_PARAMS['n_iter'],
                            p_xo=FIXED_PARAMS['p_xo'],
                            prob_const=FIXED_PARAMS['prob_const'],
                            max_depth=FIXED_PARAMS['max_depth'],
                            initializer=FIXED_PARAMS['initializer'],
                            init_depth=FIXED_PARAMS['init_depth'],
                            verbose=0, 
                            log_level=5,
                            log_path=log_path,
                            n_jobs=FIXED_PARAMS['n_jobs'],
                            test_elite=FIXED_PARAMS['test_elite']
                        )
                        print(f"        > Log saved: {scen_name} @ Split {split_idx+1}")
                    
                    except Exception as e:
                            print(f"      [Error] {ds_name} {scen_name} Split {split_idx+1}: {e}")
                            import traceback
                            traceback.print_exc()


Dataset: Toxicity

  > MC Split 1/30
    > Objectives: 2_Objs
    > Objectives: 3_Objs
    > Objectives: 5_Objs

  > MC Split 2/30
    > Objectives: 2_Objs
    > Objectives: 3_Objs
    > Objectives: 5_Objs

  > MC Split 3/30
    > Objectives: 2_Objs
    > Objectives: 3_Objs
    > Objectives: 5_Objs

  > MC Split 4/30
    > Objectives: 2_Objs
    > Objectives: 3_Objs
    > Objectives: 5_Objs

  > MC Split 5/30
    > Objectives: 2_Objs
    > Objectives: 3_Objs
    > Objectives: 5_Objs

  > MC Split 6/30
    > Objectives: 2_Objs
    > Objectives: 3_Objs
    > Objectives: 5_Objs

  > MC Split 7/30
    > Objectives: 2_Objs
    > Objectives: 3_Objs
    > Objectives: 5_Objs

  > MC Split 8/30
    > Objectives: 2_Objs
    > Objectives: 3_Objs
    > Objectives: 5_Objs

  > MC Split 9/30
    > Objectives: 2_Objs
    > Objectives: 3_Objs
    > Objectives: 5_Objs

  > MC Split 10/30
    > Objectives: 2_Objs
    > Objectives: 3_Objs
    > Objectives: 5_Objs

  > MC Split 11/30
    > Objectives: 2_

In [25]:
# #it won't be use anymore because we will fix the best hyperparameters for each dataset, scenario and objective set already in the first outer fold

# if __name__ == "__main__":
#     for ds_name, loader_func in DATASETS.items():
#         print(f"\n{'='*40}\nDataset: {ds_name}\n{'='*40}")
    
#         # Load Data (X: features, y: target)
#         X, y = loader_func(X_y=True)
    
#         # Outer CV (14 folds for train + 1 fold for test each time)
#         kf_outer = KFold(n_splits=K_OUTER, shuffle=True, random_state=RANDOM_SEED)
    
#         for outer_fold, (train_idx, test_idx) in enumerate(kf_outer.split(X, y)):
#             print(f"\n  > Outer Fold {outer_fold+1}/{K_OUTER}")
        
#             X_train_outer = X[train_idx]
#             y_train_outer = y[train_idx]
#             X_test_outer = X[test_idx]
#             y_test_outer = y[test_idx]
        
#             # Inner CV Setup
#             kf_inner = KFold(n_splits=K_INNER, shuffle=True, random_state=RANDOM_SEED)
        
#             for obj_set_name, obj_config in OBJECTIVE_SETS.items():
#                 n_objs = len(obj_config['funcs'])
#                 print(f"    > Objectives: {obj_set_name}")
            
#                 for scen_name, scen_config in SCENARIOS.items():
#                     print(f"      > Scenario: {scen_name}")
                
#                     # Hyperparameter Tuning (Inner CV)
#                     best_params = None
#                     best_inner_score = float('inf') # Min RMSE of validation
                
#                     # All possible parameter configurations
#                     current_grid = list(ParameterGrid(PARAM_GRID))
                
#                     for params in current_grid:
#                         # tournament sizes probably has a big impact but let's leave it constant for now
#                         t_sizes = [2] * n_objs

#                         inner_rmse_scores = []

#                         # Loop Inner CV: trains in (k_inner -1) folds, validates in 1 fold
#                         for inner_t_idx, inner_v_idx in kf_inner.split(X_train_outer, y_train_outer):
#                             X_in_t, y_in_t = X_train_outer[inner_t_idx], y_train_outer[inner_t_idx]
#                             X_in_v, y_in_v = X_train_outer[inner_v_idx], y_train_outer[inner_v_idx]
                        
#                             # Run MOGP (I suppress output for inner loops)
#                             try:
#                                 elite = mo_gp(
#                                     X_train=X_in_t, y_train=y_in_t,
#                                     X_test=X_in_v, y_test=y_in_v,
#                                     dataset_name=f"{ds_name}_inner",
#                                     fitness_functions=obj_config['funcs'],
#                                     minimization_flags=obj_config['flags'],
#                                     tournament_sizes=t_sizes,
#                                     elitism_strategy=scen_config['elitism_strategy'],
#                                     selector_strategy=scen_config['selector'],
#                                     survival_strategy=scen_config['survival'],
#                                     n_elites=scen_config['n_elites'],
#                                     log_level=0, verbose=0, **params # No logging for inner
#                                 )
#                                 # validation RMSE:find_mo_elites_default returns elite with test_fitness already calculated  
#                                 if elite.test_fitness is not None:
#                                     val_rmse = float(elite.test_fitness[0])
#                                 else:
#                                     val_rmse = float('inf')
#                                 inner_rmse_scores.append(val_rmse)

#                             except Exception as e:
#                                 # ############# APAGAR
#                                 # print(f"        [Inner Error] {e}") 
#                                 # import traceback
#                                 # traceback.print_exc()
#                                 # #############
#                                 # for debug
#                                 inner_rmse_scores.append(float('inf'))
                    
#                         # The winner is the one with the lowest median RMSE in validation
#                         median_rmse = np.median(inner_rmse_scores)
                    
#                         if median_rmse < best_inner_score:
#                             best_inner_score = median_rmse
#                             best_params = params.copy()
#                             best_params['tournament_sizes'] = t_sizes   # Store the calculated list 

#                     # #########DEBUG######### APAGAR
#                     # if best_params is None:
#                     #     print(f"        [WARNING] All inner runs failed for {scen_name}. Skipping outer run.")
#                     #     continue
#                     # #######################

#                     # Setup Log Directory: ./log_mo / Dataset / Scenario / Objetivos / fold_X /
#                     fold_dir = f"./log_mo/{ds_name}/{scen_name}/{obj_set_name}/fold_{outer_fold+1}"
#                     if not os.path.exists(fold_dir):
#                         os.makedirs(fold_dir)
                
#                     # Save CSV of Winner Hyperparameters
#                     # remove tournament_sizes from csv to make it cleaner
#                     if 'tournament_sizes' in best_params: del best_params['tournament_sizes'] 
                
#                     #just to have the inner score saved
#                     best_params['Median_Inner_RMSE'] = best_inner_score

#                     pd.DataFrame([best_params]).to_csv(os.path.join(fold_dir, "best_inner_params.csv"), index=False)
                
#                     print(f"        > Best Params: {best_params} (RMSE Val: {best_inner_score:.4f})")

#                     # final training with best hyperparameters on the outer fold
#                     log_path = os.path.join(fold_dir, "execution_log.csv")

#                     t_sizes_final = [2] * n_objs #needs to be recalculated here again because I dropped it before
                
#                     try:
#                         mo_gp(
#                             X_train=X_train_outer, y_train=y_train_outer,
#                             X_test=X_test_outer, y_test=y_test_outer,
#                             dataset_name=ds_name,
#                             fitness_functions=obj_config['funcs'],
#                             minimization_flags=obj_config['flags'],
#                             tournament_sizes=t_sizes_final,
#                             elitism_strategy=scen_config['elitism_strategy'],
#                             selector_strategy=scen_config['selector'],
#                             survival_strategy=scen_config['survival'],
#                             n_elites=scen_config['n_elites'],
#                             # use winner params found
#                             pop_size=best_params['pop_size'],
#                             n_iter=best_params['n_iter'],
#                             p_xo=best_params['p_xo'],
#                             prob_const=best_params['prob_const'],
#                             max_depth=best_params['max_depth'],
#                             verbose=0, 
#                             log_level=5,
#                             log_path=log_path,
#                             n_jobs=1
#                         )
#                         print(f"        > Log saved at: {log_path}")
                    
#                     except Exception as e:
#                         print(f"      [Outer Error] {e}")
#                         import traceback
#                         traceback.print_exc()

In [26]:
import os
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as sp
import scikit_posthocs as sp_post

# configuration
LOG_DIR = './log_mo'       # where logs are stored
OUTPUT_DIR = './plots_mo'  #where I'll save the plots

COLORS = {
    'Train_RMSE': '#1f77b4',
    'Test_RMSE': '#ff7f0e',
    'Std_Dev': '#9467bd',
    'Size': '#2ca02c',
    'Features': '#d62728',
    'NAO': '#17becf',
    'NAOC': '#e377c2'
}

OBJ_MAP = {
    0: "RMSE",
    1: "Size",
    2: "Features",
    3: "NAO",
    4: "NAOC"
}

In [27]:
# Transforms a column with "5.1|10.2" strings into a DataFrame of floats
def parse_fitness_column(series, log_file_name="Unknown", allow_na=False):
    # deal with NaN values first
    if series.isna().any():
        if allow_na:
            # if it is allowed, return None
            return None
        else:
            # if it is mandatory (like RMSE), NaN is a critical error
            raise ValueError(
                f"[CRITICAL ERROR] Found NaN in mandatory column in file '{log_file_name}'. "
                "This means the Elite was not evaluated correctly."
            )

    # Convert to string for uniform processing
    series_str = series.astype(str)

    # Handle the literal string "N/A" (in case pandas did not convert it to NaN)
    if (series_str == "N/A").any():
        if allow_na:
            return None
        else:
            error_idx = series_str[series_str == "N/A"].index[0]
            raise ValueError(
                f"[CRITICAL ERROR] Found 'N/A' string in file '{log_file_name}' at line {error_idx}."
            )

    # try to split and convert to float
    try:
        return series_str.str.split('|', expand=True).astype(float)
    except ValueError as e:
        raise ValueError(f"[CRITICAL] Could not convert fitness to numbers in {log_file_name}. Error: {e}")

In [28]:
# Function to load data from all logs
def load_data_for_analysis(dataset_name, objectives_key, view_mode='main'):
    target_path = os.path.join(LOG_DIR, dataset_name)
    
    if not os.path.exists(target_path):
        print(f"[ERROR] Folder not found: {target_path}")
        return None, None

    # Find all scenarios
    scenarios = [d for d in os.listdir(target_path) if os.path.isdir(os.path.join(target_path, d))]
    
    history_data = {} 
    final_rows = []   

    print(f"Logs from {dataset_name} ({objectives_key}) - View: {view_mode.upper()}")

    for scen in scenarios:
        pattern = os.path.join(target_path, scen, objectives_key, "fold_*", "execution_log.csv")
        files = glob.glob(pattern)
        
        if not files: continue
        
        history_data[scen] = []
        
        for f in files:
            try:
                df = pd.read_csv(f)
                
                # --- INDEX MAPPING (Based on MOGP Logger) ---                
                # 0: Algo, 1: UUID, 2: Dataset, 3: Seed
                # 4: Gen, 5: Main Train Fit, 6: Time, 7: Main Nodes
                # 8: Main Test Fit, 9: RMSE Elite Test Fit, 10: RMSE Elite Nodes
                # 11: Std Dev, 12: Ideal Point

                idx_offset = 4 
                
                gen_idx = 0 + idx_offset  # 4
                std_idx = 7 + idx_offset  # 11
                ideal_idx = 8 + idx_offset # 12               

                if view_mode == 'rmse':
                    # RMSE View: Main Train and RMSE Elite Test
                    train_idx = 1 + idx_offset 
                    test_idx = 5 + idx_offset
                    size_idx = 6 + idx_offset
                else:
                    # Main View: Main Train + Main Test
                    train_idx = 1 + idx_offset
                    test_idx = 4 + idx_offset
                    size_idx = 3 + idx_offset
                
                # Parse Fitness Columns    
                train_fits = parse_fitness_column(df.iloc[:, train_idx], f, allow_na=False)
                test_fits = parse_fitness_column(df.iloc[:, test_idx], f, allow_na=False)
                
                # Ideal Point (allow_na=True because it is N/A for most scenarios)
                ideal_fits = parse_fitness_column(df.iloc[:, ideal_idx], f, allow_na=True)

                # Clean DataFrame
                clean_df = pd.DataFrame({
                    'Generation': df.iloc[:, gen_idx],
                    'Std_RMSE': df.iloc[:, std_idx]
                })
                
                # Add Train Metrics
                if train_fits is not None:
                    for c in train_fits.columns:
                        clean_df[f'Train_{OBJ_MAP.get(c, c)}'] = train_fits[c]
                
                # Add Test Metrics
                if test_fits is not None:
                    for c in test_fits.columns:
                        clean_df[f'Test_{OBJ_MAP.get(c, c)}'] = test_fits[c]
                
                clean_df['Test_Size'] = df.iloc[:, size_idx]
                
                # Add Ideal Point if available
                if ideal_fits is not None:
                    clean_df['Ideal_RMSE'] = ideal_fits[0]

                history_data[scen].append(clean_df)
                
                # Data for Boxplots (Last Generation)
                last_row = clean_df.iloc[-1]
                try: fold_num = int(f.split(os.sep)[-2].split('_')[-1])
                except: fold_num = 0

                row_dict = {'Dataset': dataset_name, 'Scenario': scen, 'Fold': fold_num}
                for col in clean_df.columns:
                    if col.startswith('Train_') or col.startswith('Test_'):
                        row_dict[col] = last_row[col]
                final_rows.append(row_dict)
            except Exception as e:
                print(f"  [WARN] Could not process file {f}: {e}")

    final_df = pd.DataFrame(final_rows)
    return history_data, final_df

In [29]:
def plot_convergence(dataset_name, objectives_key, history_data, view_mode):
    save_dir = os.path.join(OUTPUT_DIR, dataset_name, objectives_key)
    if not os.path.exists(save_dir): os.makedirs(save_dir)

    for scen, folds_list in history_data.items():
        if not folds_list: continue

        sample_df = folds_list[0]
        metrics = [c.replace("Test_", "") for c in sample_df.columns if c.startswith("Test_")]
        min_len = min([len(df) for df in folds_list])
        generations = np.arange(min_len)

        fig = make_subplots(
            rows=1, cols=3,
            subplot_titles=("Performance (RMSE)", "Structural Complexity", "Pop RMSE Std Dev"),
            horizontal_spacing=0.08
        )

        def add_trace(col, name, color, idx, show_leg=True):
            if col not in sample_df.columns: return

            data = np.array([df[col].values[:min_len] for df in folds_list])
            median = np.median(data, axis=0)
            q1 = np.percentile(data, 15, axis=0)
            q3 = np.percentile(data, 85, axis=0)

            # IQR shading
            fig.add_trace(go.Scatter(
                x=np.concatenate([generations, generations[::-1]]),
                y=np.concatenate([q3, q1[::-1]]),
                fill='toself', fillcolor=color, opacity=0.15,
                line=dict(width=0), showlegend=False), row=1, col=idx)

            # Median Line
            fig.add_trace(go.Scatter(
                x=generations, y=median, mode='lines', name=name,
                line=dict(color=color)), row=1, col=idx)

        # Plot 1: Performance
        add_trace('Train_RMSE', 'Train RMSE', COLORS['Train_RMSE'], 1)
        add_trace('Test_RMSE', 'Test RMSE', COLORS['Test_RMSE'], 1)
        
        # Add Ideal Point line if it exists
        if 'Ideal_RMSE' in sample_df.columns:
             add_trace('Ideal_RMSE', 'Ideal Point', 'black', 1, show_leg=True)

        # Plot 2: Complexity
        add_trace('Test_Size', 'Size', COLORS['Size'], 2)
        for metric in [m for m in metrics if m != "RMSE" and m != "Size"]:
            color = COLORS.get(metric, '#333333')
            add_trace(f'Test_{metric}', metric, color, 2)

        # Plot 3: Std Dev
        add_trace('Std_RMSE', 'RMSE Std Dev', COLORS['Std_Dev'], 3, show_leg=False)

        fig.update_layout(
            title=f"Convergence Analysis: {scen} ({dataset_name} | {objectives_key} | {view_mode.upper()})",
            height=500, width=1400, template="plotly_white",
            legend=dict(orientation="h", y=-0.15, x=0.5, xanchor="center")
        )
        
        file_path = os.path.join(save_dir, f"convergence_{scen}_VIEW_{view_mode.upper()}.png")
        fig.write_image(file_path)
        print(f"  -> Saved convergence: {file_path}")

In [30]:
def plot_boxplots(dataset_name, objectives_key, final_df, view_mode):
    save_dir = os.path.join(OUTPUT_DIR, dataset_name, objectives_key)
    
    if final_df.empty: return

    df_melt = final_df.melt(
        id_vars=['Scenario'], 
        value_vars=['Train_RMSE', 'Test_RMSE'],
        var_name='Type', value_name='RMSE'
    )
    
    plt.figure(figsize=(12, 7))
    sns.set_style("whitegrid")

    sns.boxplot(
        data=df_melt, x='Scenario', y='RMSE', hue='Type',
        palette=[COLORS['Train_RMSE'], COLORS['Test_RMSE']],
        width=0.6, linewidth=1.2, showfliers=False 
    )
    sns.stripplot(
        data=df_melt, x='Scenario', y='RMSE', hue='Type',
        dodge=True, color='black', alpha=0.3, size=3, legend=False
    )

    plt.title(f'Final RMSE Distribution: {dataset_name} ({view_mode.upper()})', fontsize=14)
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    filename = f"boxplot_rmse_VIEW_{view_mode.upper()}.png"
    plt.savefig(os.path.join(save_dir, filename), dpi=300)
    plt.close()
    print(f"  -> Saved RMSE boxplot.")

In [31]:
def run_stats(dataset_name, objectives_key, final_df, view_mode, save_dir):
    if final_df.empty: return
    
    try:
        pivot_df = final_df.pivot(index='Fold', columns='Scenario', values='Test_RMSE')
        
        output_txt = [f"=== Stats: {dataset_name} ({objectives_key}) ===\n"]
        output_txt.append(pivot_df.describe().to_string() + "\n\n")
        
        stat, p = sp.friedmanchisquare(*[pivot_df[col] for col in pivot_df.columns])
        output_txt.append(f"Friedman Test:\nStatistic: {stat:.4f}, p-value: {p:.6f}\n")
        
        if p < 0.05:
            output_txt.append("\nSignificant differences found. Nemenyi Post-Hoc:\n")
            nemenyi = sp_post.posthoc_nemenyi_friedman(pivot_df)
            output_txt.append(nemenyi.to_string())
        
        filename = f"stats_VIEW_{view_mode.upper()}.txt"
        with open(os.path.join(save_dir, filename), "w") as f:
            f.write("".join(output_txt))
            
    except Exception as e:
        print(f"Stats Error: {e}")

In [32]:
import os
import time
import random
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import KFold, ParameterGrid
from slim_gsgp.main_mo_gp import mo_gp
# from slim_gsgp.datasets.data_loader import (
#      load_efficiency_cooling, load_ld50
# )
# DATASETS = {
#     #'Cooling': load_efficiency_cooling,
#     'Toxicity': load_ld50,
# }

In [33]:
def generate_all_plots(dataset_name, objectives_key='2_Objs', view_mode='main'):
    print(f"\n--- Processing {dataset_name} [{objectives_key}] ---")
    
    history_data, final_df = load_data_for_analysis(dataset_name, objectives_key, view_mode)
    
    if final_df is None or final_df.empty:
        print("No data found.")
        return

    plot_convergence(dataset_name, objectives_key, history_data, view_mode)
    plot_boxplots(dataset_name, objectives_key, final_df, view_mode)
    
    save_dir = os.path.join(OUTPUT_DIR, dataset_name, objectives_key)
    run_stats(dataset_name, objectives_key, final_df, view_mode, save_dir)

if __name__ == "__main__":
    for ds in DATASETS: # DATASETS dict defined in run_experiments, or hardcode list here
        for obj in ['2_Objs', '3_Objs', '5_Objs']:
            for mode in ['main', 'rmse']:
                generate_all_plots(ds, obj, mode)


--- Processing Toxicity [2_Objs] ---
Logs from Toxicity (2_Objs) - View: MAIN


KeyboardInterrupt: 

In [ ]:
#DATASETS A USAR, SENDO QUE POR AGORA QUEREMOS SÓ UM GRÁFICO PARA CADA #APAGAR
"""
load_resid_build_sale_price(X_y=True):
    Loads and returns the RESIDNAME data set (regression). Taken from https://archive.ics.uci.edu/dataset/437/residential+building+data+set

load_istanbul(X_y=True):
    Loads and returns the Istanbul data set (regression). Taken from https://docs.1010data.com/MachineLearningExamples/IstanbulDataSet.html.

load_airfoil(X_y=True):
    - Number of data instances: 1503;
    - Number of input features: 5;
    - Target's range: [103.38-140.987].

load_bike_sharing(X_y=True):
    - Number of data instances: 17389;
    - Number of input features: 13;
    - Target's range: [22, 8714].                            x

load_boston(X_y=True):
    - Number of data instances: 506;
    - Number of input features: 13;
    - Target's range: [5, 50].

 load_concrete_slump
    - Number of data instances: 103;
    - Number of input features: 7;
    - Target's range: [0, 29].

load_concrete_strength(X_y=True):
    - Number of data instances: 1005;
    - Number of input features: 8;
    - Target's range: [2.331807832, 82.5992248].

load_diabetes(X_y=True):
    - Number of data instances: 442;
    - Number of input features: 10;
    - Target's range: [25, 346].

load_efficiency_heating(X_y=True):
    - Number of data instances: 768;
    - Number of input features: 8;
    - Target's range: [6.01, 43.1].

load_efficiency_cooling(X_y=True):<-------------- usar este
    - Number of data instances: 768;
    - Number of input features: 8;
    - Target's range: [10.9, 48.03].

load_forest_fires(X_y=True):
    - Number of data instances: 513;
    - Number of input features: 43;
    - Target's range: [0.0, 6.995619625423205].

load_parkinson_updrs(X_y=True):
    - Number of data instances: 5875;                           x
    - Number of input features: 19;
    - Target's range: [7.0, 54.992].

load_ld50(X_y=True):  <-----------------usar este
    - Number of data instances: 234;
    - Number of input features: 626;
    - Target's range: [0.25, 8900.0].

load_ppb(X_y=True):                              --------------usar este
    - Number of data instances: 131;
    - Number of input features: 626;
    - Target's range: [0.5, 100.0]

load_bioav(X_y=True):                             --------------usar este
    - Number of data instances: 358;
    - Number of input features: 241;
    - Target's range: [0.4, 100.0].
"""
#APAGAR: hyperparameters
"""
(function) def mo_gp(
    X_train: Tensor,
    y_train: Tensor,
    X_test: Tensor = None,
    y_test: Tensor = None,
    dataset_name: str = None,
    pop_size: int = gp_parameters["pop_size"],
    selector_strategy: str = "nested_tournament",
    survival_strategy: str = "nsga2",
    offspring_size: int | None = None,
    n_iter: int = gp_solve_parameters["n_iter"],
    p_xo: float = gp_parameters['p_xo'],
    n_elites: int = gp_solve_parameters["n_elites"],
    max_depth: int | None = gp_solve_parameters["max_depth"],
    init_depth: int = gp_pi_init["init_depth"],
    log_path: str = None,
    seed: int = gp_parameters["seed"],
    log_level: int = gp_solve_parameters["log"],
    verbose: int = gp_solve_parameters["verbose"],
    fitness_functions: list = mo_parameters["mo_fitness_functions"],
    minimization_flags: list = mo_parameters["mo_minimization_flags"],
    tournament_sizes: list = mo_parameters["mo_tournament_sizes"],
    ideal_candidate_values: list | None = None,
    initializer: str = gp_parameters["initializer"],
    n_jobs: int = gp_solve_parameters["n_jobs"],
    prob_const: float = gp_pi_init["p_c"],
    tree_functions: list = list(FUNCTIONS.keys()),
    tree_constants: list = [float(key.replace("constant_", "").replace("_", "-")) for key in CONSTANTS],
    test_elite: bool = gp_solve_parameters["test_elite"]
) -> Any
Main function to execute the Multi-Objective Genetic Programming (MOGP) algorithm on specified datasets

Parameters
X_train: : torch.Tensor
Training input data.

y_train: : torch.Tensor
Training output data.

X_test: : torch.Tensor , optional
Testing input data.

y_test: : torch.Tensor , optional
Testing output data.

dataset_name : str, optional
Dataset name, for logging purposes

pop_size : int, optional
The population size for the genetic programming algorithm (default is 100).

selector_strategy : str, optional
The selection strategy for parent selection. Options are "nested_tournament" or "nsga2" (default is "nested_tournament").

survival_strategy : str, optional
The survival selection strategy. Options are "nsga2" or "generational" (default is "nsga2").

offspring_size : int, optional
The size of the offspring population to be generated in each generation. If None, it defaults to pop_size.

n_iter : int, optional
The number of iterations for the genetic programming algorithm (default is 100).

p_xo : float, optional
The probability of crossover in the genetic programming algorithm. Must be a number between 0 and 1 (default is 0.8).

n_elites : int, optional
The number of elites.

max_depth : int, optional
The maximum depth for the GP trees.

init_depth : int, optional
The depth value for the initial GP trees population.

log_path : str, optional
The path where is created the log directory where results are saved. Defaults to os.path.join(os.getcwd(), "log", "mo_gp.csv")

seed : int, optional
Seed for the randomness

log_level : int, optional
Level of detail to utilize in logging.

verbose : int, optional
Level of detail to include in console output.

fitness_functions : list, optional
A list of fitness function names, one for each objective. (Default is from mo_parameters)

minimization_flags : list, optional
A list of booleans indicating if each corresponding objective is for minimization (True) or maximization (False). (Default is from mo_parameters)

tournament_sizes : list, optional
A list of integers defining the tournament size for each objective during Nested Tournament Selection. (Default is from mo_parameters)

ideal_candidate_values : list, optional
A list of ideal candidate values for each objective to guide elite selection. If None, defaults uses first-objective logic.

initializer : str, optional
The strategy for initializing the population (e.g., "grow", "full", "rhh").

n_jobs : int, optional
Number of parallel jobs to run (default is 1).

prob_const : float, optional
The probability of a constant being chosen rather than a terminal in trees creation (default: 0.2).

tree_functions : list, optional
List of allowed functions that can appear in the trees. Check documentation for the available functions.

tree_constants : list, optional
List of constants allowed to appear in the trees.

test_elite : bool, optional
Whether to test the elite individual on the test set after each generation.

Returns
MultiObjectiveTree
Returns the best individual according to the tracking strategy at the last generation.

"""

"""
Tornament Size could be more "flexible"
For instance, 5% of pop_size (but has to be >1)
    t_size_val = int(params['pop_size'] * 0.05) 
    t_sizes = [max(2, t_size_val)] * n_objs
"""